In [8]:
from main import get_miner
from log_parsing import from_id_to_template
import joblib
from data_integrate import *
from pattern_miner import *
from pattern_ranker2 import *
import pandas as pd
import json

In [13]:
# Reading from a file
with open("input/root_cause_hipster.json", 'r') as file:
    rc_dict = json.load(file)

In [14]:
rc_dict

{'frontend': {'return': 'Serving product page started_GetProduct start',
  'exception': 'Placing order started_Order placed complete',
  'cpu_consumed': 'CpuUsageRate(%)',
  'cpu_contention': 'CpuUsageRate(%)'},
 'checkoutservice': {'return': 'Start charge card_Charge successfully',
  'exception': 'Start charge card_Charge successfully',
  'network_delay': 'NetworkP90(ms)',
  'cpu_contention': 'CpuUsageRate(%)',
  'cpu_consumed': 'CpuUsageRate(%)'},
 'cartservice': {'cpu_contention': 'CpuUsageRate(%)',
  'network_delay': 'NetworkP90(ms)'},
 'emailservice': {'cpu_consumed': 'CpuUsageRate(%)',
  'cpu_contention': 'CpuUsageRate(%)',
  'network_delay': 'NetworkP90(ms)'},
 'currencyservice': {'cpu_contention': 'CpuUsageRate(%)',
  'network_delay': 'NetworkP90(ms)'},
 'recommendationservice': {'cpu_consumed': 'CpuUsageRate(%)',
  'cpu_contention': 'CpuUsageRate(%)',
  'network_delay': 'NetworkP90(ms)'},
 'paymentservice': {'network_delay': 'NetworkP90(ms)',
  'cpu_contention': 'CpuUsageRate(

In [2]:
log_template_miner = get_miner()

In [3]:
res_dict = joblib.load('output/fault_free_eventgraphs.joblib')

In [4]:
res_dict2 = joblib.load('output/fault_suffering_eventgraphs.joblib')

In [5]:
res_dict2.keys()

dict_keys(['2022-08-22 03:54', '2022-08-22 03:55', '2022-08-22 04:03', '2022-08-22 04:04', '2022-08-22 04:11', '2022-08-22 04:12', '2022-08-22 04:19', '2022-08-22 04:20', '2022-08-22 04:28', '2022-08-22 04:29', '2022-08-22 04:37', '2022-08-22 04:38', '2022-08-22 04:45', '2022-08-22 04:46', '2022-08-22 04:53', '2022-08-22 04:54', '2022-08-22 05:02', '2022-08-22 05:03', '2022-08-22 05:10', '2022-08-22 05:11', '2022-08-22 05:19', '2022-08-22 05:20', '2022-08-22 05:28', '2022-08-22 05:29', '2022-08-22 05:36', '2022-08-22 05:37', '2022-08-22 05:44', '2022-08-22 05:45', '2022-08-22 05:53', '2022-08-22 05:54', '2022-08-22 06:02', '2022-08-22 06:03', '2022-08-22 06:36', '2022-08-22 06:37', '2022-08-22 06:53', '2022-08-22 06:54', '2022-08-22 07:02', '2022-08-22 07:03', '2022-08-22 07:11', '2022-08-22 07:12', '2022-08-22 07:18', '2022-08-22 07:19', '2022-08-22 07:27', '2022-08-22 07:28', '2022-08-22 07:45', '2022-08-22 07:46', '2022-08-22 07:54', '2022-08-22 07:55', '2022-08-23 12:02', '2022-08-

In [6]:
abnormal_dates = res_dict2.keys()

In [16]:
abnormal_dates = res_dict2.keys()
features_labeled = []
for normal_date in res_dict.keys():
    date = normal_date.split(' ')[0]
    fault_free_graphs = res_dict[normal_date]
    fault_free_pattern = get_pattern_support(fault_free_graphs)
    print(date)
    for abnormal_date in [d for d in abnormal_dates if d.split(' ')[0] == date]:
        print(abnormal_date)
        inject_pod = res_dict2[abnormal_date]['inject_pod']
        inject_svc = inject_pod.split("-")[0]
        inject_type = res_dict2[abnormal_date]['inject_type']
        root_cause =  rc_dict[inject_svc][inject_type].split('_')
        alarm_data_path = "input/fault_suffering"
        alarm_metric_list = get_metric_with_time(abnormal_date, alarm_data_path)
        alarm_list = generate_alarm(alarm_metric_list)
        fault_suffering_event_graphs = res_dict2[abnormal_date]['event_graphs']
        fault_suffering_pattern = get_pattern_support(fault_suffering_event_graphs)
        ranking = pattern_ranker_custom(fault_free_pattern, fault_free_graphs, fault_suffering_pattern, log_template_miner, alarm_list)
        res = { 
               'normal_time': normal_date,
               'abnormal_time': abnormal_date,
               'result_list': ranking,
               'inject_time': res_dict2[abnormal_date]['inject_time'],
               'inject_pod': inject_pod, 
               'inject_type': inject_type,
               'root_cause': root_cause
            }
        features_labeled.append(res)

2022-08-22
2022-08-22 03:54
2022-08-22 03:55
2022-08-22 04:03
2022-08-22 04:04
2022-08-22 04:11
2022-08-22 04:12
2022-08-22 04:19
2022-08-22 04:20
2022-08-22 04:28
2022-08-22 04:29
2022-08-22 04:37
2022-08-22 04:38
2022-08-22 04:45
2022-08-22 04:46
2022-08-22 04:53
2022-08-22 04:54
2022-08-22 05:02
2022-08-22 05:03
2022-08-22 05:10
2022-08-22 05:11
2022-08-22 05:19
2022-08-22 05:20
2022-08-22 05:28
2022-08-22 05:29
2022-08-22 05:36
2022-08-22 05:37
2022-08-22 05:44
2022-08-22 05:45
2022-08-22 05:53
2022-08-22 05:54
2022-08-22 06:02
2022-08-22 06:03
2022-08-22 06:36
2022-08-22 06:37
2022-08-22 06:53
2022-08-22 06:54
2022-08-22 07:02
2022-08-22 07:03
2022-08-22 07:11
2022-08-22 07:12
2022-08-22 07:18
2022-08-22 07:19
2022-08-22 07:27
2022-08-22 07:28
2022-08-22 07:45
2022-08-22 07:46
2022-08-22 07:54
2022-08-22 07:55
2022-08-23
2022-08-23 12:02
2022-08-23 12:03
2022-08-23 12:11
2022-08-23 12:12
2022-08-23 12:19
2022-08-23 12:20
2022-08-23 12:28
2022-08-23 12:29
2022-08-23 12:36
2022-08-2

In [17]:
features_labeled

[{'normal_time': '2022-08-22 03:51',
  'abnormal_time': '2022-08-22 03:54',
  'result_list': [{'events': '76_4',
    'score': 1.0,
    'deepth': 1,
    'pod': 'frontend-579b9bff58-t2dbm',
    'resource_alert': 'CpuUsageRate(%)',
    'events_actual': '76_197'}],
  'inject_time': '2022-08-22 03:53:54',
  'inject_pod': 'frontend-579b9bff58-t2dbm',
  'inject_type': 'cpu_contention',
  'root_cause': ['CpuUsageRate(%)']},
 {'normal_time': '2022-08-22 03:51',
  'abnormal_time': '2022-08-22 03:55',
  'result_list': [{'events': '76_4',
    'score': 1.0,
    'deepth': 1,
    'pod': 'frontend-579b9bff58-t2dbm',
    'resource_alert': 'CpuUsageRate(%)',
    'events_actual': '76_197'},
   {'events': '104_106',
    'score': 0.7028169014084507,
    'deepth': 2,
    'pod': 'currencyservice-cf787dd48-vpjrd'}],
  'inject_time': '2022-08-22 03:53:54',
  'inject_pod': 'frontend-579b9bff58-t2dbm',
  'inject_type': 'cpu_contention',
  'root_cause': ['CpuUsageRate(%)']},
 {'normal_time': '2022-08-22 03:51',
 

In [18]:
# Convert to DataFrame
df = pd.DataFrame(features_labeled)
# locate templates with err or fail in string, treat them special (dont lose on ranking)

In [19]:
df.head()

,normal_time,abnormal_time,result_list,inject_time,inject_pod,inject_type,root_cause
0,2022-08-22 03:51,2022-08-22 03:54,"[{'events': '76_4', 'score': 1.0, 'deepth': 1,...",2022-08-22 03:53:54,frontend-579b9bff58-t2dbm,cpu_contention,[CpuUsageRate(%)]
1,2022-08-22 03:51,2022-08-22 03:55,"[{'events': '76_4', 'score': 1.0, 'deepth': 1,...",2022-08-22 03:53:54,frontend-579b9bff58-t2dbm,cpu_contention,[CpuUsageRate(%)]
2,2022-08-22 03:51,2022-08-22 04:03,"[{'events': '120_198', 'score': 1.0, 'deepth':...",2022-08-22 04:02:07,frontend-579b9bff58-t2dbm,return,"[Serving product page started, GetProduct start]"
3,2022-08-22 03:51,2022-08-22 04:04,"[{'events': '120_198', 'score': 1.0, 'deepth':...",2022-08-22 04:02:07,frontend-579b9bff58-t2dbm,return,"[Serving product page started, GetProduct start]"
4,2022-08-22 03:51,2022-08-22 04:11,"[{'events': '76_4', 'score': 1.0, 'deepth': 1,...",2022-08-22 04:10:20,frontend-579b9bff58-t2dbm,cpu_consumed,[CpuUsageRate(%)]


In [20]:
df.to_csv('output/features_labeled_v3.csv', index=False)